### Step 01: ML pipeline

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import re
import os
import pickle

# Ensure output directories exist
plots_dir = 'plots'
models_dir = 'models'
os.makedirs(plots_dir, exist_ok=True)
os.makedirs(models_dir, exist_ok=True)

# Load dataset
data = pd.read_excel('../dataset/england_dataset.xlsx')

# Data preprocessing
# Drop non-numerical or identifier columns
X = data.drop(columns=['Area Code', 'Area Name', 'Area Type [Note 3]', 'index health'])
y = data['index health']

# Clean feature column names: keep only letters, digits, and underscores
X.columns = [re.sub(r'[^a-zA-Z0-9]', '_', col) for col in X.columns]
# Ensure column names start with a letter (required by LightGBM)
X.columns = ['f_' + col if col[0].isdigit() else col for col in X.columns]

# Print cleaned column names for debugging
print("Cleaned feature column names:")
print(X.columns.tolist())

# Handle missing values (fill with 0 for simplicity)
X = X.fillna(0)

# Validate sample count
print(f"\nTotal number of samples in dataset: {len(data)}")
if len(data) != 2387:
    raise ValueError("The dataset should contain 2387 samples. Please check your file!")

# Define manual 5-fold split (477, 477, 477, 477, 479)
fold_sizes = [477, 477, 477, 477, 479]
fold_indices = []
start_idx = 0
for fold_size in fold_sizes:
    fold_indices.append(range(start_idx, start_idx + fold_size))
    start_idx += fold_size

# Define models
models = {
    'Gradient Boosting': GradientBoostingRegressor(random_state=42),
    'Random Forest': RandomForestRegressor(random_state=42),
    'XGBoost': XGBRegressor(random_state=42),
    'LightGBM': LGBMRegressor(random_state=42, verbose=-1)
}

# Initialize results storage and best models
results = {model_name: {'train_rmse': [], 'train_r2': [], 'test_rmse': [], 'test_r2': []} for model_name in models}
best_models = {model_name: {'model': None, 'test_rmse': float('inf')} for model_name in models}

# Perform manual 5-fold cross-validation
for fold_idx, test_indices in enumerate(fold_indices, 1):
    print(f"\nFold {fold_idx}")
    
    test_idx = list(test_indices)
    train_idx = [i for i in range(len(data)) if i not in test_idx]
    
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    print(f"  Training samples: {len(X_train)}, Test samples: {len(X_test)}")
    
    for model_name, model in models.items():
        # Clone model to ensure fresh instance
        model_instance = type(model)(**model.get_params())
        model_instance.fit(X_train, y_train)
        
        y_train_pred = model_instance.predict(X_train)
        train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
        train_r2 = r2_score(y_train, y_train_pred)
        
        y_test_pred = model_instance.predict(X_test)
        test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
        test_r2 = r2_score(y_test, y_test_pred)
        
        results[model_name]['train_rmse'].append(train_rmse)
        results[model_name]['train_r2'].append(train_r2)
        results[model_name]['test_rmse'].append(test_rmse)
        results[model_name]['test_r2'].append(test_r2)
        
        print(f"{model_name} - Fold {fold_idx}:")
        print(f"  Train RMSE: {train_rmse:.4f}, Train R²: {train_r2:.4f}")
        print(f"  Test RMSE: {test_rmse:.4f}, Test R²: {test_r2:.4f}")
        
        # Update best model if test RMSE is lower
        if test_rmse < best_models[model_name]['test_rmse']:
            best_models[model_name]['model'] = model_instance
            best_models[model_name]['test_rmse'] = test_rmse
            print(f"  Updated best {model_name} model in fold {fold_idx} with Test RMSE: {test_rmse:.4f}")

# Save best models using pickle
try:
    for model_name in best_models:
        model_filename = os.path.join(models_dir, f'best_{model_name.lower().replace(" ", "_")}.pkl')
        with open(model_filename, 'wb') as f:
            pickle.dump(best_models[model_name]['model'], f)
        print(f"Saved best {model_name} model to {model_filename}")
except Exception as e:
    print(f"Error saving models: {str(e)}")

# Compute average results
avg_results = {model_name: {} for model_name in models}
for model_name in models:
    avg_results[model_name]['avg_train_rmse'] = np.mean(results[model_name]['train_rmse'])
    avg_results[model_name]['avg_train_r2'] = np.mean(results[model_name]['train_r2'])
    avg_results[model_name]['avg_test_rmse'] = np.mean(results[model_name]['test_rmse'])
    avg_results[model_name]['avg_test_r2'] = np.mean(results[model_name]['test_r2'])

# Print average results
print("\nAverage Results for Each Model:")
for model_name in models:
    print(f"\n{model_name}:")
    print(f"  Avg Train RMSE: {avg_results[model_name]['avg_train_rmse']:.4f}")
    print(f"  Avg Train R²: {avg_results[model_name]['avg_train_r2']:.4f}")
    print(f"  Avg Test RMSE: {avg_results[model_name]['avg_test_rmse']:.4f}")
    print(f"  Avg Test R²: {avg_results[model_name]['avg_test_r2']:.4f}")

# Plot RMSE comparison
plt.figure(figsize=(12, 6))
x = np.arange(len(models))
width = 0.2

train_rmse_means = [avg_results[model]['avg_train_rmse'] for model in models]
plt.bar(x - width, train_rmse_means, width, label='Avg Train RMSE', color='skyblue')

test_rmse_means = [avg_results[model]['avg_test_rmse'] for model in models]
plt.bar(x, test_rmse_means, width, label='Avg Test RMSE', color='salmon')

plt.xlabel('Models')
plt.ylabel('RMSE')
plt.title('Average RMSE Comparison Across Models')
plt.xticks(x, models.keys())
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(plots_dir, 'rmse_comparison.png'))
plt.close()

# Plot R² comparison
plt.figure(figsize=(12, 6))

train_r2_means = [avg_results[model]['avg_train_r2'] for model in models]
plt.bar(x - width, train_r2_means, width, label='Avg Train R²', color='lightgreen')

test_r2_means = [avg_results[model]['avg_test_r2'] for model in models]
plt.bar(x, test_r2_means, width, label='Avg Test R²', color='orange')

plt.xlabel('Models')
plt.ylabel('R²')
plt.title('Average R² Comparison Across Models')
plt.xticks(x, models.keys())
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(plots_dir, 'r2_comparison.png'))
plt.close()

# Plot RMSE per fold for each model
for model_name in models:
    plt.figure(figsize=(10, 5))
    folds = range(1, 6)
    plt.plot(folds, results[model_name]['train_rmse'], marker='o', label='Train RMSE', color='blue')
    plt.plot(folds, results[model_name]['test_rmse'], marker='o', label='Test RMSE', color='red')
    plt.xlabel('Fold')
    plt.ylabel('RMSE')
    plt.title(f'{model_name} RMSE per Fold')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(plots_dir, f'{model_name.lower().replace(" ", "_")}_rmse_folds.png'))
    plt.close()

# Plot R² per fold for each model
for model_name in models:
    plt.figure(figsize=(10, 5))
    folds = range(1, 6)
    plt.plot(folds, results[model_name]['train_r2'], marker='o', label='Train R²', color='green')
    plt.plot(folds, results[model_name]['test_r2'], marker='o', label='Test R²', color='orange')
    plt.xlabel('Fold')
    plt.ylabel('R²')
    plt.title(f'{model_name} R² per Fold')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(plots_dir, f'{model_name.lower().replace(" ", "_")}_r2_folds.png'))
    plt.close()

print("\nGenerated chart files:")
print(f"- {os.path.join(plots_dir, 'rmse_comparison.png')}")
print(f"- {os.path.join(plots_dir, 'r2_comparison.png')}")
for model_name in models:
    print(f"- {os.path.join(plots_dir, f'{model_name.lower().replace(' ', '_')}_rmse_folds.png')}")
    print(f"- {os.path.join(plots_dir, f'{model_name.lower().replace(' ', '_')}_r2_folds.png')}")

Cleaned feature column names:
['Healthy_People_Domain', 'Difficulties_in_daily_life__Pe_', 'Disability__Pe1_', 'Frailty__Pe1_', 'Mental_health__Pe_', 'Children_s_social__emotional_and_mental_health__Pe2_', 'Mental_health_conditions__Pe2_', 'Self_harm__Pe2_', 'Suicides__Pe2_', 'Mortality__Pe_', 'Avoidable_mortality__Pe3_', 'Infant_mortality__Pe3_', 'Life_expectancy__Pe3_', 'Mortality_from_all_causes__Pe3_', 'Personal_well_being__Pe_', 'Activities_in_life_are_worthwhile__Pe4_', 'Feelings_of_anxiety__Pe4_', 'Happiness__Pe4_', 'Life_satisfaction__Pe4_', 'Physical_health_conditions__Pe_', 'Cancer__Pe5_', 'Cardiovascular_conditions__Pe5_', 'Dementia__Pe5_', 'Diabetes__Pe5_', 'Kidney_and_liver_disease__Pe5_', 'Musculoskeletal_conditions__Pe5_', 'Respiratory_conditions__Pe5_', 'Healthy_Lives_Domain', 'Behavioural_risk_factors__L_', 'Alcohol_misuse__L1_', 'Drug_misuse__L1_', 'Healthy_eating__L1_', 'Physical_activity__L1_', 'Sedentary_behaviour__L1_', 'Sexually_transmitted_infections__L1_', 'Smo

### Step 02: SHAP explainers

In [4]:
X_test.shape

(479, 73)

Load each model and generate SHAP Explainer for Global Explanations

In [18]:
import shap
import pickle

d:\POLYTECH_2025\health_index_v2\hi_env2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
def load_stored_models(filename, inputs, compute_shap = False):
    with open(f'models/{filename}.pkl', 'rb') as f:
        model = pickle.load(f)
    predictions = model.predict(inputs)

    if compute_shap:
        # Initialize SHAP explainer
        explainer = shap.Explainer(model, inputs)
        # Compute SHAP values
        shap_values = explainer(inputs)
        return predictions, shap_values
    
    return predictions, None
        

In [20]:
models = ['best_gradient_boosting', 'best_random_forest', 'best_xgboost', 'best_lightgbm']

Calculate the predictions and SHAP values for each model

In [22]:
outs = {}
shap_outs = {}

for model in models:
    preds, shaps = load_stored_models(model, X_test, compute_shap= True)

    if preds is not None:
        outs[model] = preds
        shap_outs[model] = shaps
    else:
        print("Error loading computation")

100%|===================| 477/479 [00:37<00:00]        

Generate the SHAP Global Explaners graphics

In [23]:
for model in shap_outs:
    try:
        if shap_outs[model] is not None:
            print(f"----- SHAP Graphics for Model {model} ------")
            # Ensure X_test is a DataFrame for plotting
            plot_inputs = X_test if isinstance(X_test, pd.DataFrame) else pd.DataFrame(X_test)
            
            # Set consistent figure size
            plt.figure(figsize=(10, 6))

            # 1. Bar Plot (Feature Importance)
            shap.plots.bar(shap_outs[model], max_display=12, show=False)
            bar_filename = os.path.join(plots_dir, f'shap_bar_{model}.png')
            plt.savefig(bar_filename, dpi=300, bbox_inches='tight')
            plt.close()
            print(f"Saved SHAP bar plot for {model} to {bar_filename}")

            # 2. Summary Plot (Beeswarm)
            plt.figure(figsize=(10, 6))
            shap.summary_plot(shap_outs[model], plot_inputs, show=False)
            summary_filename = os.path.join(plots_dir, f'shap_summary_{model}.png')
            plt.savefig(summary_filename, dpi=300, bbox_inches='tight')
            plt.close()
            print(f"Saved SHAP summary plot for {model} to {summary_filename}")

            # 3. Heatmap Plot
            plt.figure(figsize=(10, 6))
            shap.plots.heatmap(shap_outs[model], max_display=12, show=False)
            heatmap_filename = os.path.join(plots_dir, f'shap_heatmap_{model}.png')
            plt.savefig(heatmap_filename, dpi=300, bbox_inches='tight')
            plt.close()
            print(f"Saved SHAP heatmap plot for {model} to {heatmap_filename}")
        else:
            print(f"No SHAP values available for {model}, skipping plots")
    except Exception as e:
        print(f"Error generating SHAP plots for {model}: {str(e)}")

----- SHAP Graphics for Model best_gradient_boosting ------
Saved SHAP bar plot for best_gradient_boosting to plots\shap_bar_best_gradient_boosting.png
Saved SHAP summary plot for best_gradient_boosting to plots\shap_summary_best_gradient_boosting.png
Saved SHAP heatmap plot for best_gradient_boosting to plots\shap_heatmap_best_gradient_boosting.png
----- SHAP Graphics for Model best_random_forest ------
Saved SHAP bar plot for best_random_forest to plots\shap_bar_best_random_forest.png
Saved SHAP summary plot for best_random_forest to plots\shap_summary_best_random_forest.png
Saved SHAP heatmap plot for best_random_forest to plots\shap_heatmap_best_random_forest.png
----- SHAP Graphics for Model best_xgboost ------
Saved SHAP bar plot for best_xgboost to plots\shap_bar_best_xgboost.png
Saved SHAP summary plot for best_xgboost to plots\shap_summary_best_xgboost.png
Saved SHAP heatmap plot for best_xgboost to plots\shap_heatmap_best_xgboost.png
----- SHAP Graphics for Model best_lightgb